In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import preprocessing
from tensorflow.keras.models import Model, load_model
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from konlpy.tag import Okt
import numpy as np
from PIL import Image
from wordcloud import STOPWORDS
import seaborn as sns
import time
import re


# 현재 시간을 출력해주는 라이브러리
now = time.localtime()
yesterday = "%04d-%02d-%02d" % (now.tm_year, now.tm_mon, now.tm_mday) # 자정 12시에 실행해야 하므로 어제 날짜로 변수 생성
print(yesterday)



# 문자 색 변경 함수
def yellow_color_func(word, font_size, position,orientation,random_state=None, **kwargs):
    return("hsl({:d},{:d}%, {:d}%)".format(np.random.randint(60,65),np.random.randint(70,100),np.random.randint(45,55)))

def red_color_func(word, font_size, position,orientation,random_state=None, **kwargs):
    return("hsl({:d},{:d}%, {:d}%)".format(np.random.randint(0,10),np.random.randint(70,100),np.random.randint(45,70)))

def pink_color_func(word, font_size, position,orientation,random_state=None, **kwargs):
    return("hsl({:d},{:d}%, {:d}%)".format(np.random.randint(300,313),np.random.randint(70,100),np.random.randint(45,80)))

def blue_color_func(word, font_size, position,orientation,random_state=None, **kwargs):
    return("hsl({:d},{:d}%, {:d}%)".format(np.random.randint(220,230),np.random.randint(70,100),np.random.randint(55,80)))



# mysql 데이터베이스로 연결
import pymysql
id_list = []

db = None

try:
    db = pymysql.connect(
        host='project-db-stu.ddns.net',         # 호스트명
        port=3307,                              # 포트번호
        user='com',                             # mysql ID
        passwd='12345',                         # mysql password
        db='campus_E_3_1022',                   # 데이터베이스명
        charset='utf8'                          # 인코딩
    )
    print('DB 연결 성공')
    
    
    # 하루 채팅했던 회원들의 ID들을 리스트에 저장
    with db.cursor(pymysql.cursors.DictCursor) as cursor:
        
        
        # 어제 날짜기준으로 채팅내역이 있는 ID들을 출력
        sql = '''
            select distinct mb_id from chattings WHERE DATE_FORMAT(chat_date, '%Y-%m-%d') = '{}'
        '''.format(yesterday)
        cursor.execute(sql)
        results = cursor.fetchall()
        
        
        # ID들을 리스트에 저장
        for i in results:
            id_list.append(i['mb_id'])
    print(id_list)
    
    
    # 
    for k in id_list:
        
        sentence = ''
        sentence_c = ''
        sentence_s = ''
        word = ''
        num = 0
        num_c = 0

        
        # 유저 채팅 횟수
        with db.cursor(pymysql.cursors.DictCursor) as cursor:
            
            
            # 어제 회원이 채팅했던 횟수를 변수에 저장
            sql = '''
                select count(chat_user) from chattings WHERE DATE_FORMAT(chat_date, '%Y-%m-%d') = '{}' and mb_id = '{}'
            '''.format(yesterday, k)
            cursor.execute(sql)
            results = cursor.fetchall()
            num = results[0]['count(chat_user)']
        print(num)

        
        # 유저 채팅 가져오기
        with db.cursor(pymysql.cursors.DictCursor) as cursor:
            
            
            # 어제 회원이 채팅했던 내역을을 출력
            sql = '''
                select chat_user from chattings WHERE mb_id = '{}' and DATE_FORMAT(chat_date, '%Y-%m-%d') = '{}'
            '''.format(k, yesterday)
            cursor.execute(sql)
            results = cursor.fetchall()
            
            
            # 채팅 내역들을 한 문장으로 합쳐 변수에 저장
            # 채팅횟수만큼 반복
            for i in range(num):
                if results[i]['chat_user'] != None:
                    sentence += results[i]['chat_user']+' '

                
            # 영어 특수문자 제거
            sentence = re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '', sentence)
            corpus = [preprocessing.text.text_to_word_sequence(sentence)]
            tokenizer = preprocessing.text.Tokenizer()
            tokenizer.fit_on_texts(corpus)
            sequences = tokenizer.texts_to_sequences(corpus)

            MAX_SEQ_LEN = 15 # 최대 패딩(토큰 수) 설정
            padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding='post')

            
            # CNN모델 불러오기
            
            model = load_model('cnn_model_0.94.h5')

            
            
            pick = [0]
            predict = model.predict(padded_seqs[pick])
            predict_class = tf.math.argmax(predict, axis=1)
            emotion = predict_class.numpy()[0]
            print('감정 예측 클래스 :', emotion)
            # 감정분석 완료
            # 0 - 기쁨
            # 1 - 분노
            # 2 - 슬픔
            # 3 - 신남

        # 챗봇 채팅횟수
        with db.cursor(pymysql.cursors.DictCursor) as cursor:
            sql = '''
                select count(chat_chatbot) from chattings WHERE DATE_FORMAT(chat_date, '%Y-%m-%d') = '{}' and mb_id = '{}'
            '''.format(yesterday, k)
            cursor.execute(sql)
            results = cursor.fetchall()
            num_c = results[0]['count(chat_chatbot)']
        print(num_c)

        # 챗봇 채팅 가져오기
        with db.cursor(pymysql.cursors.DictCursor) as cursor:
            sql = '''
                select chat_chatbot from chattings WHERE mb_id = '{}' and DATE_FORMAT(chat_date, '%Y-%m-%d') = '{}'
            '''.format(k, yesterday)
            cursor.execute(sql)
            results = cursor.fetchall()
            for i in range(num):
                if results[i]['chat_chatbot'] != None:
                    sentence_c += results[i]['chat_chatbot']+' '


        sentence_s = sentence + ' ' + sentence_c
        okt = Okt()
        nouns = okt.nouns(sentence_s)
        for i in range(len(nouns)):
            word += nouns[i]+ ' '

        print(sentence)
        if emotion == 0:
            mask = np.array(Image.open('circle.jpg'))
            stopwords = set(STOPWORDS)
            #stopwords.add('said')

            wordcloud = WordCloud(font_path='NanumGothic.otf', max_font_size=30, min_font_size = 10, max_words=200,
                                  background_color='white', mask=mask, stopwords=stopwords).generate(word)
            plt.figure(figsize=(8,8)) #이미지 사이즈 지정
            plt.imshow(wordcloud.recolor(color_func=yellow_color_func, random_state=3), interpolation='lanczos') #이미지의 부드럽기 정도
            plt.axis('off')
            plt.show()
        elif emotion == 1:
            mask = np.array(Image.open('circle.jpg'))
            stopwords = set(STOPWORDS)
            #stopwords.add('said')

            wordcloud = WordCloud(font_path='NanumGothic.otf', max_font_size=30, min_font_size = 10, max_words=200,
                                  background_color='white', mask=mask, stopwords=stopwords).generate(word)
            plt.figure(figsize=(8,8)) #이미지 사이즈 지정
            plt.imshow(wordcloud.recolor(color_func=red_color_func, random_state=3), interpolation='lanczos') #이미지의 부드럽기 정도
            plt.axis('off')
            plt.show()
        elif emotion == 2:
            mask = np.array(Image.open('circle.jpg'))
            stopwords = set(STOPWORDS)
            #stopwords.add('said')

            wordcloud = WordCloud(font_path='NanumGothic.otf', max_font_size=30, min_font_size = 10, max_words=200,
                                  background_color='white', mask=mask, stopwords=stopwords).generate(word)
            plt.figure(figsize=(8,8)) #이미지 사이즈 지정
            plt.imshow(wordcloud.recolor(color_func=blue_color_func, random_state=3), interpolation='lanczos') #이미지의 부드럽기 정도
            plt.axis('off')
            plt.show()
        elif emotion == 3:
            mask = np.array(Image.open('circle.jpg'))
            stopwords = set(STOPWORDS)
            #stopwords.add('said')

            wordcloud = WordCloud(font_path='NanumGothic.otf', max_font_size=30, min_font_size = 10, max_words=200,
                                  background_color='white', mask=mask, stopwords=stopwords).generate(word)
            plt.figure(figsize=(8,8)) #이미지 사이즈 지정
            plt.imshow(wordcloud.recolor(color_func=pink_color_func, random_state=3), interpolation='lanczos') #이미지의 부드럽기 정도
            plt.axis('off')
            plt.show()




        wordcloud.to_file("C:\\Users\\bigdata02\\git\\CESCO\\HOY\\src\\main\\webapp\\resources\\img\\{}_{}.png".format(k,yesterday))


        #데이터 삽입
        with db.cursor() as cursor:
            sql = '''
            insert into mycalendar(mb_id, mood_type, wordcloud) values('{}', {}, '{}_{}.png')
            '''.format(user, emotion, user, yesterday)
            cursor.execute(sql)
            db.commit()
        print('감정종류 입력 완료')

    
except Exception as e:
    print(e) # db 연결 실패 시 오류 내용 출력

finally:
    if db is not None: # db가 연결된 경우에만 접속 닫기 시도
        db.close()
    print('DB 연결 닫기 성공')

2021-11-06
DB 연결 성공
['0201', '123', '456', 'admin1']
1
감정 예측 클래스 : 3
1
 
We need at least 1 word to plot a word cloud, got 0.
DB 연결 닫기 성공
